December 03, 2024

Goal: Figure out how to properly configure `qcec.verify` to setup the benchmark experiments. Let's first load up a large, complex circuit from T-opt benchmarks so that the resulting changes are observable.

In [ ]:
cd ..

this time, let's consider the `mod_adder_1024.qasm` circuit:

In [ ]:
mod_adder_1024_path = "../feynman/benchmarks/qasm/mod_adder_1024.qasm"

from qiskit import QuantumCircuit
mod_adder_circ = QuantumCircuit.from_qasm_file(mod_adder_1024_path)
mod_adder_circ.draw()

**Important note:** We observe that unlike the `qft_4` circuit which had all gates in `Clifford + T`, the adder circuit is represented using `{Toffoli, H}`

Now, let's initialize and setup a `Configuration` object:

In [ ]:
from mqt import qcec
from mqt.qcec import Configuration
from mqt.qcec.configuration import augment_config_from_kwargs
config = Configuration()

In [ ]:
config_dict = {
    # application
    "alternating_scheme": "proportional",
    "simulation_scheme": "proportional",
    
    # execution
    "run_zx_checker": False,
    "timeout": 60.,
    
    # functionality
    
    # optimizations
    "elide_permutations": False,
    "fuse_single_qubit_gates": False,
    "reconstruct_swaps": False,
    "reorder_operations": False,
    "transform_dynamic_circuit": False,
    
    # simulation
}

In [ ]:
# update the configuration with the dictionary
augment_config_from_kwargs(config, config_dict)

Now, let's attempt to transpile the above circuit and perform equivalence checking

In [ ]:
from keys import IBMQ_API
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum", token=IBMQ_API, instance="ibm-q/open/main")
ibm_backend = service.backend("ibm_sherbrooke")
mod_adder_circ_hw = transpile(mod_adder_circ, ibm_backend)

In [ ]:
mod_adder_circ_hw.draw()

Let's add measurements to both the original and the transpiled circuits:

In [ ]:
mod_adder_circ.measure_all()
mod_adder_circ_hw.measure_all()

And now let's verify the equivalence:

In [ ]:
qcec.verify(mod_adder_circ, mod_adder_circ_hw, config)